In [59]:
from sympy import *
from sympy.matrices import BlockMatrix, Matrix, zeros
import sympy as sp


In [159]:
Ts = 0.03
dt = 0.01
foot_height_keep_STF = 0.2
N = int(Ts/dt)
print('Nb steps = ', N)
nb_total_variables_per_coordinate = int(3 * N) # px, vx, zx
print('nb_total_variables_per_coordinate', nb_total_variables_per_coordinate)
nb_total_variables = 3 * nb_total_variables_per_coordinate # x, y, z
print(nb_total_variables, 'variables')

Nb steps =  3
nb_total_variables_per_coordinate 9
27 variables


In [160]:
variables = []
coord = ['x', 'y', 'z']
for c in coord:
    for i in range(N):
        p = symbols(f'p{c}{i}')
        v = symbols(f'v{c}{i}')
        a = symbols(f'a{c}{i}')
        variables.extend([p, v, a])
pva_vector = Matrix(variables)
print('------------------- Optimization Variable -------------------')
print('pva_vector = ', len(pva_vector), ' variables')
for i in range(3):
    pprint(pva_vector.T[i*nb_total_variables_per_coordinate:(i+1)*nb_total_variables_per_coordinate])


------------------- Optimization Variable -------------------
pva_vector =  27  variables
[px₀, vx₀, ax₀, px₁, vx₁, ax₁, px₂, vx₂, ax₂]
[py₀, vy₀, ay₀, py₁, vy₁, ay₁, py₂, vy₂, ay₂]
[pz₀, vz₀, az₀, pz₁, vz₁, az₁, pz₂, vz₂, az₂]


In [209]:
from sympy import BlockMatrix, BlockDiagMatrix, MutableMatrix, ZeroMatrix

print('------------------- P Mat -------------------')
opt_weight_desired_pos = 10
opt_weight_desired_vel = 1
zero_matrix_3_by_3 = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])

block = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
block[-1, -1] = 1
P_matrix = zeros(nb_total_variables, nb_total_variables)

# put the 3x3 block on the diagonals of P_matrix
for i in range(nb_total_variables_per_coordinate):
    P_matrix[i*3:(i+1)*3, i*3:(i+1)*3] = block
    
P_matrix[nb_total_variables_per_coordinate - 3, nb_total_variables_per_coordinate - 3] = opt_weight_desired_pos
P_matrix[2*nb_total_variables_per_coordinate - 3, 2*nb_total_variables_per_coordinate - 3] = opt_weight_desired_pos
P_matrix[3*nb_total_variables_per_coordinate - 3, 3*nb_total_variables_per_coordinate - 3] = opt_weight_desired_pos

P_matrix[nb_total_variables_per_coordinate - 2, nb_total_variables_per_coordinate - 2] = opt_weight_desired_vel
P_matrix[2*nb_total_variables_per_coordinate - 2, 2*nb_total_variables_per_coordinate - 2] = opt_weight_desired_vel
P_matrix[3*nb_total_variables_per_coordinate - 2, 3*nb_total_variables_per_coordinate - 2] = opt_weight_desired_vel

vPv = pva_vector.T * P_matrix * pva_vector

p_N_des = 0.2
v_N_des = 0.0
q = zeros(nb_total_variables, 1)
q[nb_total_variables_per_coordinate - 3] = -2*opt_weight_desired_pos*p_N_des
q[2*nb_total_variables_per_coordinate - 3] = -2*opt_weight_desired_pos*p_N_des
q[3*nb_total_variables_per_coordinate - 3] = -2*opt_weight_desired_pos*p_N_des

q[nb_total_variables_per_coordinate - 2] = -2*opt_weight_desired_vel*v_N_des
q[2*nb_total_variables_per_coordinate - 2] = -2*opt_weight_desired_vel*v_N_des
q[3*nb_total_variables_per_coordinate - 2] = -2*opt_weight_desired_vel*v_N_des

objective_function = vPv + q.T * pva_vector
print('Objective function')
pprint(objective_function)


------------------- P Mat -------------------
Objective function
⎡   2      2      2      2      2      2      2      2      2         2       
⎣ax₀  + ax₁  + ax₂  + ay₀  + ay₁  + ay₂  + az₀  + az₁  + az₂  + 10⋅px₂  - 4.0⋅

            2                   2                2                2           
px₂ + 10⋅py₂  - 4.0⋅py₂ + 10⋅pz₂  - 4.0⋅pz₂ + vx₂  - 0.4⋅vx₂ + vy₂  - 0.4⋅vy₂ 

     2          ⎤
+ vz₂  - 0.4⋅vz₂⎦


In [162]:

A_eq_pos_vel_des = sp.zeros(6, nb_total_variables)

A_eq_pos_vel_des[0, 0] = 1
A_eq_pos_vel_des[1, 1] = 1

A_eq_pos_vel_des[2, nb_total_variables_per_coordinate] = 1
A_eq_pos_vel_des[3, nb_total_variables_per_coordinate + 1] = 1

A_eq_pos_vel_des[4, 2*nb_total_variables_per_coordinate] = 1
A_eq_pos_vel_des[5, 2*nb_total_variables_per_coordinate + 1] = 1

result_vector = A_eq_pos_vel_des*pva_vector
print('------------------- Boundary Points Equality Contraints -------------------')
print('A_eq_pos_vel_desired * v = ')
pprint(result_vector)

------------------- Boundary Points Equality Contraints -------------------
A_eq_pos_vel_desired * v = 
⎡px₀⎤
⎢   ⎥
⎢vx₀⎥
⎢   ⎥
⎢py₀⎥
⎢   ⎥
⎢vy₀⎥
⎢   ⎥
⎢pz₀⎥
⎢   ⎥
⎣vz₀⎦


In [167]:
dt_symbol = symbols('dt')
block_dynamics = zeros(2, 6)
block_dynamics[0, 0] = 1
block_dynamics[0, 1] = dt_symbol
block_dynamics[0, 3] = -1
block_dynamics[1, 1] = 1
block_dynamics[1, 2] = dt_symbol
block_dynamics[1, 4] = -1
A_dynamics_per_coordinate = zeros(2 * N - 2, 3 * N)

j = 0
for i in range(int((2 * N - 2)/2)):
    A_dynamics_per_coordinate[j : j + 2, i * 3: i * 3 + 6] = block_dynamics
    j = j + 2

pva_vector_x_comp = pva_vector[0:nb_total_variables_per_coordinate, 0]

size_rows_A = A_dynamics_per_coordinate.shape[0]
size_cols_A = A_dynamics_per_coordinate.shape[1]
A_dynamics = zeros(3 * size_rows_A, nb_total_variables)

for i in range(3):
    A_dynamics[i * size_rows_A : i * size_rows_A + size_rows_A, i * size_cols_A : i * size_cols_A + size_cols_A] = A_dynamics_per_coordinate

mult = A_dynamics * pva_vector
print('------------------- Dynamic Contraints -------------------')
pprint(mult)

------------------- Dynamic Contraints -------------------
⎡dt⋅vx₀ + px₀ - px₁⎤
⎢                  ⎥
⎢ax₀⋅dt + vx₀ - vx₁⎥
⎢                  ⎥
⎢dt⋅vx₁ + px₁ - px₂⎥
⎢                  ⎥
⎢ax₁⋅dt + vx₁ - vx₂⎥
⎢                  ⎥
⎢dt⋅vy₀ + py₀ - py₁⎥
⎢                  ⎥
⎢ay₀⋅dt + vy₀ - vy₁⎥
⎢                  ⎥
⎢dt⋅vy₁ + py₁ - py₂⎥
⎢                  ⎥
⎢ay₁⋅dt + vy₁ - vy₂⎥
⎢                  ⎥
⎢dt⋅vz₀ + pz₀ - pz₁⎥
⎢                  ⎥
⎢az₀⋅dt + vz₀ - vz₁⎥
⎢                  ⎥
⎢dt⋅vz₁ + pz₁ - pz₂⎥
⎢                  ⎥
⎣az₁⋅dt + vz₁ - vz₂⎦


In [180]:
block = zeros(2, 3)
block[0, 1] = 1
block[1, 2] = 1
A_limits = zeros(nb_total_variables_per_coordinate * 2, nb_total_variables)

for i in range(nb_total_variables_per_coordinate):
    A_limits[2*i:2*(i+1), 3*i:3*(i+1)] = block

pprint((A_limits*pva_vector).T)

[vx₀  ax₀  vx₁  ax₁  vx₂  ax₂  vy₀  ay₀  vy₁  ay₁  vy₂  ay₂  vz₀  az₀  vz₁  az
₁  vz₂  az₂]


In [188]:
A_total = Matrix.vstack(A_eq_pos_vel_des,
                    A_dynamics,
                    A_limits)

print('A * PVA_VECTOR')
pprint(A_total*pva_vector)

A * PVA_VECTOR
⎡       px₀        ⎤
⎢                  ⎥
⎢       vx₀        ⎥
⎢                  ⎥
⎢       py₀        ⎥
⎢                  ⎥
⎢       vy₀        ⎥
⎢                  ⎥
⎢       pz₀        ⎥
⎢                  ⎥
⎢       vz₀        ⎥
⎢                  ⎥
⎢dt⋅vx₀ + px₀ - px₁⎥
⎢                  ⎥
⎢ax₀⋅dt + vx₀ - vx₁⎥
⎢                  ⎥
⎢dt⋅vx₁ + px₁ - px₂⎥
⎢                  ⎥
⎢ax₁⋅dt + vx₁ - vx₂⎥
⎢                  ⎥
⎢dt⋅vy₀ + py₀ - py₁⎥
⎢                  ⎥
⎢ay₀⋅dt + vy₀ - vy₁⎥
⎢                  ⎥
⎢dt⋅vy₁ + py₁ - py₂⎥
⎢                  ⎥
⎢ay₁⋅dt + vy₁ - vy₂⎥
⎢                  ⎥
⎢dt⋅vz₀ + pz₀ - pz₁⎥
⎢                  ⎥
⎢az₀⋅dt + vz₀ - vz₁⎥
⎢                  ⎥
⎢dt⋅vz₁ + pz₁ - pz₂⎥
⎢                  ⎥
⎢az₁⋅dt + vz₁ - vz₂⎥
⎢                  ⎥
⎢       vx₀        ⎥
⎢                  ⎥
⎢       ax₀        ⎥
⎢                  ⎥
⎢       vx₁        ⎥
⎢                  ⎥
⎢       ax₁        ⎥
⎢                  ⎥
⎢       vx₂        ⎥
⎢                  ⎥
⎢       ax₂        

In [198]:
# Limits
p0_desired = 0.0
v0_desired = 0.0

l_boundary_pts_per_coordinate = Matrix([p0_desired, v0_desired])
u_boundary_pts_per_coordinate = Matrix([p0_desired, v0_desired])


l_boundary_pts = Matrix.vstack(l_boundary_pts_per_coordinate, l_boundary_pts_per_coordinate, l_boundary_pts_per_coordinate)
u_boundary_pts = Matrix.vstack(u_boundary_pts_per_coordinate, u_boundary_pts_per_coordinate, u_boundary_pts_per_coordinate)

l_boundary_pts

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])

In [206]:
v_max = 1000.0
a_max = 3000.0
print(N)
l_limits_per_coordinate = zeros(2 * N, 1)
u_limits_per_coordinate = zeros(2 * N, 1)
print(l_limits_per_coordinate.shape)
l_limits_per_coordinate[0::2] = -v_max
l_limits_per_coordinate[1::2] = -a_max
u_limits_per_coordinate[0::2] = v_max
u_limits_per_coordinate[1::2] = a_max

l_limits = np.vstack([l_limits_per_coordinate] * 3)
u_limits = np.vstack([u_limits_per_coordinate] * 3)

3
(6, 1)


ValueError: unexpected value: -1000.0